In [1]:
import pandas as pd, numpy as np

# FCC Internet Speed Data

Internet providers report speeds to the FCC via form 477. 

* Data: https://www.fcc.gov/general/broadband-deployment-data-fcc-form-477
* Glossary: https://www.fcc.gov/general/explanation-broadband-deployment-data

# Part 0: Importing data

In [37]:
tracts = pd.read_csv("data/tracts_to_towns.csv")
tracts.head()

,tract,town
0,9001010101,Greenwich
1,9001010102,Greenwich
2,9001010201,Greenwich
3,9001010202,Greenwich
4,9001010300,Greenwich


In [3]:
providers = pd.read_excel("data/Provider_List_Jun2015_v3.xlsx")
providers.head()

,FRN,ProviderName,StateAbbr,TechCode,records,blocks
0,10827,Veracity Networks,UT,50,4008,4008
1,12542,SERVICE ONE CABLE,LA,41,726,726
2,12781,"NEU VENTURES, INC.",TX,41,921,921
3,12781,"NEU VENTURES, INC.",TX,70,1093,1093
4,12849,"MUTUAL COMMUNICATIONS SERVICES, INC.",IA,41,21,21


In [70]:
ct_speeds = pd.read_csv("data/CT-Fixed-Jun2015-v3.csv")
print len(ct_speeds["BlockCode"].unique())
ct_speeds.head()

66103


,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp
0,192364,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051013000,40,1,110,20,1,105,20
1,192365,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011006,40,1,110,20,1,105,20
2,192366,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011038,40,1,110,20,1,105,20
3,192367,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011020,40,1,110,20,1,105,20
4,192368,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011018,40,1,110,20,1,105,20


# Part 0-1: Joining data

I'm going to add a town column to the ct_speeds dataframe, translating each tract to the town its a part of.

In [5]:
def list_of(x, n):
    return map(lambda a: x, range(n))

print list_of(False, 5) + (list_of(False, 5))
print list_of(False, 10)


[False, False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False, False]


In [6]:
# Clean town names
tracts["town"] = tracts.apply(lambda x: x["town"].title().strip(), axis=1)
tracts.head()


,tract,town
0,9001010101,Greenwich
1,9001010102,Greenwich
2,9001010201,Greenwich
3,9001010202,Greenwich
4,9001010300,Greenwich


In [7]:

# Add a column of census tracts, shorter than the block codes
def code_to_tract(code):
    sample_tract = "9001010101"
    tract_length = len(sample_tract)
    # return int(str(code)[:tract_length])
    return int('{0:d}'.format(int(code))[:tract_length])

code_to_tract(90117051013000)

ct_speeds["tract"] = ct_speeds.apply(lambda x: code_to_tract(x["BlockCode"]), axis=1)
ct_speeds.head()

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp,tract
0,192364,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051013000,40,1,110,20,1,105,20,9011705101
1,192365,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011006,40,1,110,20,1,105,20,9011705101
2,192366,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011038,40,1,110,20,1,105,20,9011705101
3,192367,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011020,40,1,110,20,1,105,20,9011705101
4,192368,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011018,40,1,110,20,1,105,20,9011705101


In [8]:
ct_speeds = ct_speeds.merge(tracts, left_on="tract", right_on="tract")
ct_speeds.head()


,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp,tract,town
0,192364,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051013000,40,1,110,20,1,105,20,9011705101,Stonington
1,192365,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011006,40,1,110,20,1,105,20,9011705101,Stonington
2,192366,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011038,40,1,110,20,1,105,20,9011705101,Stonington
3,192367,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011020,40,1,110,20,1,105,20,9011705101,Stonington
4,192368,10990,8797680,"Thames Valley Communications, Inc.",Thames Valley Communications,"Thames Valley Communications, Inc.",130576,"Thames Valley Communications, Inc.",CT,90117051011018,40,1,110,20,1,105,20,9011705101,Stonington


In [9]:
# checksum
print ct_speeds["town"].value_counts().sum()
print len(ct_speeds.index)

427788
427788


# Part 3: Analysis

In [10]:
def gen_report(town_df):
    print "There are " + str(len(town_df["BlockCode"].unique())) + " census blocks and "\
    + str(len(town_df["tract"].unique())) + " tracts in __."
    print str(len(town_df["DBAName"].unique())) + " companies provide service."
    
    print "Advertised max speeds range from "\
    + str(town_df["MaxAdUp"].min()) + "/" + str(town_df["MaxAdDown"].min())\
    + " to "\
    + str(town_df["MaxAdUp"].max()) + "/" + str(town_df["MaxAdDown"].max())
    
    return town_df

def town_report(town_name):
    print "Generating report for " + town_name
    return gen_report(ct_speeds[ct_speeds["town"] == town_name.title()])

town_report("Greenwich").head()

Generating report for Greenwich
There are 1246 census blocks and 15 tracts in __.
23 companies provide service.
Advertised max speeds range from 0.0/0.0 to 100.0/101.0


,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp,tract,town
10181,199770,11012,10296853,"Broadview Networks Holdings, Inc.","Broadview Networks, Inc.","Broadview Networks Holdings, Inc.",130166,"Broadview Networks Holdings, Inc.",CT,90010102013008,10,0,0,0,1,3,0.786,9001010201,Greenwich
10182,1562991,11593,18589226,"Lightower Fiber Networks I, LLC (formerly know...",Lightower,LTS Group Holdings LLC,131095,LTS Group Holdings LLC,CT,90010102011000,50,0,0,0,1,1000,1000.000,9001010201,Greenwich
10183,1562992,11593,18589226,"Lightower Fiber Networks I, LLC (formerly know...",Lightower,LTS Group Holdings LLC,131095,LTS Group Holdings LLC,CT,90010102011011,50,0,0,0,1,1000,1000.000,9001010201,Greenwich
10184,1562993,11593,18589226,"Lightower Fiber Networks I, LLC (formerly know...",Lightower,LTS Group Holdings LLC,131095,LTS Group Holdings LLC,CT,90010102011012,50,0,0,0,1,1000,1000.000,9001010201,Greenwich
10185,1562994,11593,18589226,"Lightower Fiber Networks I, LLC (formerly know...",Lightower,LTS Group Holdings LLC,131095,LTS Group Holdings LLC,CT,90010102011018,50,0,0,0,1,1000,1000.000,9001010201,Greenwich


In [11]:
def summary_table(df, col):
    return ct_speeds.groupby(col)\
    .agg({
            "MaxAdDown":max,
            "MaxAdUp":max,
            "MaxCIRDown":max,
            "MaxCIRUp":max,
            "DBAName":lambda x: x.nunique(),
#             "BlockCode":lambda x: x.nunique(),   
         })\
    .reset_index()

town_speeds = summary_table(ct_speeds, "town")

town_speeds.head()

,town,DBAName,MaxAdDown,MaxAdUp,MaxCIRDown,MaxCIRUp
0,Ansonia,12,150,35,1000,1000
1,Ashford,7,100,5,1000,1000
2,Avon,14,150,20,1000,1000
3,Barkhamsted,7,100,5,1000,1000
4,Beacon Falls,9,150,20,1000,1000


#### Top business speeds: you can get 1 Gb everywhere

Every town offers 1000 Mbps up/down for businesses.

MaxCIRDown/Up - Maximum contractual downstream[/upstream] bandwidth offered by the provider in the block for Business service (filer directed to report 0 if the contracted service is sold on a "best efforts" basis without a guaranteed data-throughput rate)



In [12]:
town_speeds["MaxCIRUp"].value_counts()

1000    167
Name: MaxCIRUp, dtype: int64

In [13]:
town_speeds["MaxCIRDown"].value_counts()

1000    167
Name: MaxCIRDown, dtype: int64

#### Max advertised speed per town

In [14]:
town_speeds[["town","MaxAdUp","MaxAdDown"]].sort_values(by="MaxAdDown")

,town,MaxAdUp,MaxAdDown
83,Montville,6,75
149,Waterford,6,75
135,Sterling,6,75
66,Killingly,6,75
91,New London,6,75
43,Eastford,5,100
60,Hampton,5,100
75,Mansfield,6,100
92,New Milford,6,100
94,Newmilford,6,100


In [15]:
summary_table(ct_speeds[(ct_speeds["Consumer"] == 1)\
                       &(ct_speeds["MaxAdDown"] > 0)],
              "BlockCode")["MaxAdDown"].value_counts()

150    28114
101    14407
15     10486
100     8593
75      2549
0        543
110      528
18       465
55       414
Name: MaxAdDown, dtype: int64

In [74]:
town_speeds.sort_values(by="MaxAdDown")

,town,DBAName,MaxAdDown,MaxAdUp,MaxCIRDown,MaxCIRUp
83,Montville,9,75,6,1000,1000
149,Waterford,16,75,6,1000,1000
135,Sterling,8,75,6,1000,1000
66,Killingly,11,75,6,1000,1000
91,New London,15,75,6,1000,1000
43,Eastford,8,100,5,1000,1000
60,Hampton,8,100,5,1000,1000
75,Mansfield,9,100,6,1000,1000
92,New Milford,12,100,6,1000,1000
94,Newmilford,9,100,6,1000,1000


#### How much "competition?"

There were an average of 12 providers, with as many as 24 and as few as 7 providers in each CT town.

In [16]:
town_speeds["DBAName"].describe()

count    167.000000
mean      12.616766
std        4.235161
min        7.000000
25%        9.000000
50%       12.000000
75%       15.500000
max       24.000000
Name: DBAName, dtype: float64

In [17]:
summary_table(ct_speeds, "tract")["DBAName"].describe()

count    829.000000
mean      10.382388
std        2.622778
min        6.000000
25%        8.000000
50%       10.000000
75%       12.000000
max       20.000000
Name: DBAName, dtype: float64

#### Businesses have a lot of choices
That's a surprisingly high number of providers, when I know that in practice it's nearly impossible as a consumer to get more than one company to provide internet access to an address, let alone 12. What if we exclude business providers?

In [18]:
summary_table(ct_speeds[ct_speeds["Consumer"] == 1], "BlockCode")["DBAName"].describe()

count    66099.000000
mean         6.015991
std          1.398897
min          1.000000
25%          5.000000
50%          6.000000
75%          7.000000
max         18.000000
Name: DBAName, dtype: float64

#### Still higher than expected...

I still didn't expect to see an average of 6 companies providing consumer internet acess in a given census block.

Despite what the data has checked off, I don't believe these are all truly consumer ISPs. https://www.megapath.com/ describes itself as a business internet provider.

In [19]:
def list_providers(town_name):
    return ct_speeds[(ct_speeds["town"] == town_name.title())
                    & (ct_speeds["Consumer"] == 1)]["DBAName"].unique()

list_providers("Bethel")



array(['Frontier Communications Corporation', 'Cablevision', 'Comcast',
       'MegaPath Corporation', 'dishNET Satellite Broadband LLC',
       'Global Capacity LLC', 'ViaSat Inc', 'HughesNet', 'Skycasters',
       'Charter Communications Inc'], dtype=object)

In [20]:
providers[providers["StateAbbr"] == "CT"]\
.drop_duplicates(subset=["ProviderName"]).sort_values(by="blocks", ascending=False)

,FRN,ProviderName,StateAbbr,TechCode,records,blocks
64,1568880,GCI Communication Corp.,CT,60,64389,64389
5587,12369286,"HNS License Sub, LLC",CT,60,64389,64389
7159,18756155,"VSAT Systems, LLC",CT,60,64389,64389
3849,4963088,"ViaSat, Inc.",CT,60,64389,64389
4422,6797849,"Fiber Technologies Networks, L.L.C.",CT,50,32726,32726
1242,3576352,Frontier Communications Corporation,CT,10,23499,23499
2565,3768165,"COMCAST CABLE COMMUNICATIONS, LLC",CT,42,23000,23000
7276,19027440,CSC Holdings LLC,CT,42,14652,14652
2227,3746468,"Charter Communications, Inc.",CT,42,8926,8926
529,1834696,"Cox Communications, Inc",CT,42,6024,6024


In [21]:
ct_consumer_speeds = ct_speeds[ct_speeds["Consumer"] == 1]

def dbas_in(provider_name, col="tract", df=ct_consumer_speeds):
    return ct_consumer_speeds[(df["DBAName"] == provider_name)]\
.drop_duplicates(subset=[col])

def count_in(provider_name,col="tract"):
    return len(dbas_in('Frontier Communications Corporation',
                       col=col)[col].index)


print count_in('Frontier Communications Corporation',"BlockCode")
print count_in('Frontier Communications Corporation',"tract")
print count_in('Frontier Communications Corporation',"town")

42041
818
167


In [22]:
def test():
    ct_consumer_speeds = ct_speeds[ct_speeds["Consumer"] == 1]
    ct_non_consumer_speeds = ct_speeds[ct_speeds["Consumer"] == 0]
    
    print "Consumer records:" + str(len(ct_consumer_speeds.index))
    print "Non-consumer records:" + str(len(ct_non_consumer_speeds.index))
test()

Consumer records:322480
Non-consumer records:105308


In [23]:
def blocks_in_state(dba):
    return ct_consumer_speeds[(ct_consumer_speeds["DBAName"] == dba)]

def blocks_in_town(dba):
    df = blocks_in_state(dba)
    return df[(df["town"] == 'Bethel')]

def tracts_in_state(dba):
    return blocks_in_state(dba).drop_duplicates(subset=["tract"])

def towns_in_state(dba):
    return blocks_in_state(dba).drop_duplicates(subset=["town"])

def max_upload(df):
    return df["MaxAdUp"].max()

def max_download(df):
    return df["MaxAdDown"].max()

def row_count(df):
    return len(df.index)

row_count(blocks_in_state('Charter Communications Inc'))

8926

In [24]:
def all_providers ():
    provider_names = []
    provider_count_in_towns = []
    provider_count_in_blocks = []
    ret = []
    for dba in ct_consumer_speeds\
    .drop_duplicates(subset="DBAName")["DBAName"]:
        ret.append([
                dba,
                row_count(towns_in_state(dba)),
                row_count(tracts_in_state(dba)),
                row_count(blocks_in_state(dba)),
                max_upload(blocks_in_state(dba)),
                max_download(blocks_in_state(dba))
            ])
        
#         provider_names.append(dba)
#         provider_count_in_towns.append(count_in(dba, "town"))
#         provider_count_in_blocks.append(count_in(dba,"BlockCode"))
#     return [provider_names,provider_count_in_towns,provider_count_in_blocks]
    return pd.DataFrame(ret,columns=["Provider",
                                     "Consumer Towns",
                                     "Consumer Tracts",
                                     "Consumer Blocks",
                                    "Max Upstream",
                                    "Max Downstream"])

all_providers().sort_values(by="Consumer Blocks",
                           ascending=False)


,Provider,Consumer Towns,Consumer Tracts,Consumer Blocks,Max Upstream,Max Downstream
1,Frontier Communications Corporation,167,818,71349,6.000,55.0
4,ViaSat Inc,167,829,64389,3.000,12.0
5,HughesNet,167,829,64389,2.000,15.0
6,Skycasters,167,829,64389,1.300,2.0
2,Comcast,106,463,22348,20.000,150.0
7,Cablevision,49,242,14648,35.000,101.0
11,Charter Communications Inc,51,93,8926,5.000,100.0
12,Cox Communications,30,121,5810,20.000,150.0
14,Metrocast,20,43,2574,5.000,75.0
0,Thames Valley Communications,3,16,1529,20.000,110.0


In [25]:
ct_speeds[ct_speeds["DBAName"]=="Frontier Communications Corporation"]["MaxAdUp"].max()
ct_speeds[ct_speeds["DBAName"]=="Frontier Communications Corporation"]["MaxAdDown"].max()

55.0

#### Differences by blocks and tracks

In [26]:
blocks = ct_speeds["BlockCode"].drop_duplicates().sort_values().to_frame()
tracts_data = ct_speeds["tract"].drop_duplicates().sort_values().to_frame()

In [27]:
def tract_df (tract):
    return ct_speeds[ct_speeds["tract"] == tract]
def block_df(block):
    return ct_speeds[ct_speeds["BlockCode"] == block]

def max_download_in_block(block):
    return max_download(block_df(block))

def max_download_in_tract(tract):
    return max_download(tract_df(tract))

In [28]:
tracts_data["MaxAdDown"] = tracts_data.apply(lambda x: max_download_in_tract(x["tract"]),axis=1)
tracts_data["MaxAdDown"].value_counts()

150    553
101    199
100     50
75      26
55       1
Name: MaxAdDown, dtype: int64

In [41]:
blocks["MaxAdDown"] = blocks.apply(lambda x: max_download_in_block(x["BlockCode"]),axis=1)
blocks["MaxAdUp"] = blocks.apply(lambda x: max_download_in_block(x["BlockCode"]),axis=1)
blocks["MaxAdDown"].value_counts()

150    28114
101    14407
15     10486
100     8593
75      2549
0        543
110      528
18       465
55       414
Name: MaxAdDown, dtype: int64

In [30]:
row_count(blocks[blocks["MaxAdDown"] < 25])

11494

In [31]:
row_count(blocks[blocks["MaxAdDown"] >= 25])

54605

In [32]:
blocks.columns

Index([u'BlockCode', u'MaxAdDown'], dtype='object')

In [33]:
#blocks.apply(lambda x: "{0:d}".format(int(x["BlockCode"])),axis=1).to_frame()
blocks["tract"] = blocks.apply(lambda x: code_to_tract(x["BlockCode"]),axis=1).to_frame()
blocks.head()

,BlockCode,MaxAdDown,tract
423588,90010101011000,101,9001010101
423589,90010101011001,101,9001010101
423755,90010101011002,101,9001010101
423653,90010101011003,101,9001010101
423824,90010101011004,101,9001010101


In [34]:
#blocks[blocks["MaxAdDown"] < 25].to_csv("output/slow_blocks.csv")

In [35]:
#tracts["town"] = tracts.apply(lambda x: x["town"].title().strip(), axis=1)


In [42]:
blocks = blocks.merge(tracts,left_on="tract",right_on="tract")
blocks.head()

,BlockCode,MaxAdDown,tract,town_x,town_y,MaxAdUp,town
0,90010101011000,101,9001010101,Greenwich,Greenwich,101,Greenwich
1,90010101011001,101,9001010101,Greenwich,Greenwich,101,Greenwich
2,90010101011002,101,9001010101,Greenwich,Greenwich,101,Greenwich
3,90010101011003,101,9001010101,Greenwich,Greenwich,101,Greenwich
4,90010101011004,101,9001010101,Greenwich,Greenwich,101,Greenwich


In [78]:
# Identify each town's percentage of slow blocks
# A slow block fails to meet the definition of broadband of 25 Mbps down / 3 up
def pct_slow(df):
    total_blocks = len((df["BlockCode"].unique()))
    #print "total blocks:", total_blocks
    tmpdf = df[(df["MaxAdUp"] > 0) | (df["MaxAdDown"] > 0)]
    tmpdf = tmpdf[(df["MaxAdUp"] < 3) | (df["MaxAdDown"] < 25)]
    slow_blocks = len(tmpdf.index)
    #print "slow_blocks",slow_blocks
    try:
        return [slow_blocks, total_blocks, slow_blocks * 100 / total_blocks]
    except:
        return [0,0,0]

def pct_slow_town(town_name):
    return pct_slow(blocks[blocks["town"] == town_name])

def slow_towns():
    ret = []
    for t in tracts["town"].unique():
        ret.append([t] + pct_slow_town(t))
    return pd.DataFrame(ret, columns=["town","slow blocks","total blocks","percent slow"])

print slow_towns()[["town","percent slow","slow blocks","total blocks"]]\
.sort_values(by="percent slow", ascending=False)\
.to_csv(sep="\t",index=False)

/Library/Python/2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


town	percent slow	slow blocks	total blocks
 North Stonington	52	82	155
Salisbury	45	132	293
Hartland	43	41	94
 Norfolk	42	126	298
 Chester	42	68	161
 Madison	42	207	482
Sharon	41	108	260
 Haddam	38	127	332
 Lyme	38	64	165
 Portland	38	89	234
Windsor	37	187	497
 Preston	37	69	186
 Cromwell	37	99	265
 Salem	37	47	124
 Franklin	36	27	73
Bloomfield	36	193	525
 Colchester	36	120	332
 East Hampton	35	96	274
East Granby	35	48	135
 Middlebury	35	94	263
Windsor Locks	35	102	285
 Essex	34	58	168
 Bozrah	34	43	126
Marlborough	34	54	156
 Branford	33	174	526
Berlin	33	139	421
 New London	32	163	503
Rocky Hill	31	100	317
Suffield	31	28	88
 Middlefield	31	72	229
 Middletown	31	211	662
 Deep River	30	39	126
Avon	30	107	351
Farmington	30	134	439
 East Haddam	30	104	341
North Canaan	30	36	119
 Killingly	30	169	554
Burlington	29	52	176
Glastonbury	29	177	600
 North Haven	28	144	507
 North Branford	28	81	289
 Stafford	28	135	468
East Hartford	28	209	746
Hartford	27	318	1151
 Sterling	27	33	118
 Waterford	

In [79]:
slow_pct_towns = slow_towns()[["town","percent slow","slow blocks","total blocks"]]
slow_pct_towns.sort_values("percent slow", ascending=False)

,town,percent slow,slow blocks,total blocks
132,North Stonington,52,82,155
57,Salisbury,45,132,293
23,Hartland,43,41,94
78,Norfolk,42,126,298
87,Chester,42,68,161
111,Madison,42,207,482
58,Sharon,41,108,260
85,Haddam,38,127,332
121,Lyme,38,64,165
82,Portland,38,89,234


In [80]:
print slow_pct_towns["total blocks"].sum()
print slow_pct_towns["slow blocks"].sum()
print slow_pct_towns["slow blocks"].sum() * 100 / slow_pct_towns["total blocks"].sum()

66099
10951
16
